In [1]:
cd ../

/Users/arbaaz/Downloads/mabe/mabe_code


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from data_utils import (
    get_index, 
    get_file_names,
    get_paths,
    get_parqt,
    get_col_names, 
    sleep
)


path = "/Users/arbaaz/Downloads/mabe/ds"
get_file_names(path)

['.DS_Store',
 'sample_submission.csv',
 'test.csv',
 'test_tracking',
 'train.csv',
 'train_annotation',
 'train_tracking']

In [27]:
get_file_names(path+"/train_annotation")

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [195]:
import pandas as pd


train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [54]:
not_in_test = [
    "MABe22_keypoints",
    "MABe22_movies",
    "CalMS21_supplemental",
    "CalMS21_task1",
    "CalMS21_task2",
    "CRIM13"
]

In [205]:
train_meta = train_metadata.query("~lab_id.isin(@not_in_test)")
len(train_meta)

368

In [143]:
# Checking if all ids of mouses are exlusive or not 

id_cols = get_col_names("_id")

for id, grp in train_meta.groupby("lab_id"):
    all_ids_lsts = []
    
    for col in id_cols:
        id_lst = grp[col].unique().tolist()
        all_ids_lsts.append(id_lst)

    all_ids = [x for lst in all_ids_lsts for x in lst]
    
    if not len(set(all_ids)) == len(all_ids):
        print(f"Error in {id}")

Error in GroovyShrew


In [144]:
shrew = train_meta.groupby("lab_id").get_group("GroovyShrew")

all_ids_lsts = []

for col in id_cols:
    id_lst = shrew[col].unique().tolist()
    all_ids_lsts.append(id_lst)

all_ids_lsts

[[13.0, 18.0, 4.0, 15.0, 9.0, 19.0, 7.0],
 [16.0, 14.0, 8.0, 3.0, 6.0, 2.0, 1.0, 12.0, 4.0, 5.0],
 [nan],
 [nan]]

In [157]:
pretrain_labs = [
    "MABe22_keypoints",
    "MABe22_movies",
]

In [158]:
pretrain_meta = train_metadata.query("lab_id.isin(@pretrain_labs)")
pretrain_meta["lab_id"].unique()

array(['MABe22_keypoints', 'MABe22_movies'], dtype=object)

In [159]:
# Checking if all ids of mouses are exlusive or not 

id_cols = get_col_names("_id")

for id, grp in pretrain_meta.groupby("lab_id"):
    all_ids_lsts = []
    
    for col in id_cols:
        id_lst = grp[col].unique().tolist()
        all_ids_lsts.append(id_lst)

    all_ids = [x for lst in all_ids_lsts for x in lst]
    
    if not len(set(all_ids)) == len(all_ids):
        print(f"Error in {id}")

In [164]:
calcrim = [
    "CalMS21_supplemental",
    "CalMS21_task1",
    "CalMS21_task2",
    "CRIM13"
]

In [203]:
calcrim_meta = train_metadata.query("lab_id.isin(@calcrim)")
calcrim_meta["lab_id"].unique()

array(['CRIM13', 'CalMS21_supplemental', 'CalMS21_task1', 'CalMS21_task2'],
      dtype=object)

In [ ]:
# Checking if all ids of mouses are exlusive or not 

id_cols = get_col_names("_id")

for id, grp in calcrim_meta.groupby("lab_id"):
    all_ids_lsts = []
    
    for col in id_cols:
        id_lst = grp[col].unique().tolist()
        all_ids_lsts.append(id_lst)

    all_ids = [x for lst in all_ids_lsts for x in lst]
    
    if not len(set(all_ids)) == len(all_ids):
        print(f"Error in {id}")

In [216]:
from collections import defaultdict


all_ids_dict = defaultdict(set)

# check global ids of all mouses unique or not

for lab_id, grp in train_metadata.groupby("lab_id"):
    for col in id_cols:
        all_ids_dict[col].update(grp[col].unique())

In [221]:
for v in all_ids_dict.values():
    for x in v:
        print(x)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
nan
nan
nan
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
nan
12.0
nan
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
24.0
25.0
26.0
nan
28.0
nan
30.0
31.0
32.0
33.0
35.0
36.0
38.0
40.0
42.0
44.0
47.0
48.0
50.0
53.0
nan
nan
nan
nan
nan
nan
13.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
29.0
31.0
32.0
33.0
34.0
35.0
nan
nan
38.0
39.0
40.0
42.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
44.0
45.0
46.0
47.0
48.0
51.0
52.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
